In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

from os.path import join

plt.style.use(["seaborn", "thesis"])

/home/jo/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Fetch Dataset 

In [38]:
from SCFInitialGuess.utilities.dataset import extract_triu_batch, AbstractDataset
from sklearn.model_selection import train_test_split
from pyscf.scf import hf

data_path = "../../dataset/TSmall_sto3g"
postfix = "TSmall_sto3g"
dim = 26
#data_path = "../../../butadien/data/"
#postfix = ""
#dim = 26


def split(x, y, ind):
    return x[:ind], y[:ind], x[ind:], y[ind:]

S = np.load(join(data_path, "S" + postfix + ".npy"))
P = np.load(join(data_path, "P" + postfix + ".npy"))
F = np.load(join(data_path, "F" + postfix + ".npy"))

index = np.load(join(data_path, "index" + postfix + ".npy"))


ind = int(0.8 * len(index))

molecules = np.load(join(data_path, "molecules" + postfix + ".npy"))
molecules = (molecules[:ind], molecules[ind:])



s_triu_norm, mu, std = AbstractDataset.normalize(S)

h_test = [hf.get_hcore(mol.get_pyscf_molecule()) for mol in molecules[1]]

s_train, f_train, s_test, f_test = split(S.reshape(-1, dim, dim), F.reshape(-1, dim, dim), ind)

# Pure blocks

i will now set all other elements zero

## Utilities

In [4]:
from SCFInitialGuess.utilities.constants import number_of_basis_functions as N_BASIS

mol = molecules[0][0]

mask = np.zeros((dim, dim))

current_dim = 0
for atom in mol.species:
    
    # calculate block range
    index_start = current_dim
    current_dim += N_BASIS["sto-3g"][atom] 
    index_end = current_dim
    
    # calculate logical vector
    L = np.arange(dim)
    L = np.logical_and(index_start <= L, L < index_end)
    
    m = np.logical_and.outer(L, L)
    mask = np.logical_or(mask, m)
    
#mask

In [58]:
from pyscf.scf import hf

def density_from_fock(f, s, mol):
    
    mo_energy, mo_coeff = hf.eig(f, s)
    mo_occ = hf.get_occ(hf.SCF(mol), mo_energy, mo_coeff)
    
    return hf.make_rdm1(mo_coeff, mo_occ)

def density_from_fock_batch(f_batch):
    p = []
    for (s, f, mol) in zip(s_test, f_batch, molecules[1]):
        p.append(density_from_fock(f, s, mol.get_pyscf_molecule()))
    return np.array(p)

In [32]:
from SCFInitialGuess.utilities.dataset import reconstruct_from_triu

def extract_pure(p):
    p_batch = p.copy()
    p_batch[np.logical_not(mask)] = 0.0
    return p_batch

f_pure = np.array(list(map(extract_pure, f_test))).astype("float64")

In [33]:
p_pure = density_from_fock_batch(s_test, f_test)

(26, 26) (26, 26)


LinAlgError: the leading minor of order 1 of 'b' is not positive definite. The factorization of 'b' could not be completed and no eigenvalues or eigenvectors were computed.

In [14]:
from SCFInitialGuess.utilities.analysis import make_results_str, measure_all_quantities
from SCFInitialGuess.utilities.dataset import StaticDataset

N_ELECTRONS = 30

dataset = StaticDataset(
    train=(s_train, f_train),
    validation=(None, None),
    test=(s_test, f_test),
    mu=mu,
    std=std
)


## Pure Block Analysis 

In [21]:
from SCFInitialGuess.utilities.analysis import mf_initializer as mf_initializer


print(make_results_str(measure_all_quantities(
    p_pure,
    dataset,
    molecules[1],
    N_ELECTRONS,
    mf_initializer,
    dim,
    is_triu=False,
    is_dataset_triu=False,
    s=S[ind:]
)))

ValueError: cannot reshape array of size 201 into shape (26,26)

# Embedding the Blocks In Classical Guesses

In [39]:
from pyscf.scf import hf

p_sad = np.array([
    hf.init_guess_by_atom(mol.get_pyscf_molecule()) for mol in molecules[1]
]).astype("float64")

In [45]:
def fock_from_density(p_batch):
    f = []
    for p, s, h, mol in zip(p_batch, s_test, h_test, molecules[1]):
        f.append(hf.get_fock(h, s, hf.get_veff(mol.get_pyscf_molecule(), p), p))
    return f

f_sad = fock_from_density(p_sad)

In [52]:
from SCFInitialGuess.utilities.dataset import reconstruct_from_triu

def embedd(x):
    p = x[1].copy()
    p[mask] = (x[0].copy())[mask]
    return p

f_embedded_sad = np.array(list(map(embedd, zip(f_test, f_sad))))

In [59]:
p_embedded_sad = density_from_fock_batch(f_embedded_sad)

## SAD

In [61]:
print(make_results_str(measure_all_quantities(
    p_embedded_sad,
    dataset,
    molecules[1],
    N_ELECTRONS,
    mf_initializer,
    dim,
    is_triu=False,
    is_dataset_triu=False,
    s=S[ind:]
)))

[ ] 2018-10-04 20:05:09: Iteration calculation: 0
[ ] 2018-10-04 20:05:10: Iteration calculation: 1
[ ] 2018-10-04 20:05:11: Iteration calculation: 2
[ ] 2018-10-04 20:05:13: Iteration calculation: 3
[ ] 2018-10-04 20:05:14: Iteration calculation: 4
[ ] 2018-10-04 20:05:16: Iteration calculation: 5
[ ] 2018-10-04 20:05:17: Iteration calculation: 6
[ ] 2018-10-04 20:05:18: Iteration calculation: 7
[ ] 2018-10-04 20:05:19: Iteration calculation: 8
[ ] 2018-10-04 20:05:20: Iteration calculation: 9
[ ] 2018-10-04 20:05:22: Iteration calculation: 10
[ ] 2018-10-04 20:05:23: Iteration calculation: 11
[ ] 2018-10-04 20:05:24: Iteration calculation: 12
[ ] 2018-10-04 20:05:26: Iteration calculation: 13
[ ] 2018-10-04 20:05:28: Iteration calculation: 14
[ ] 2018-10-04 20:05:29: Iteration calculation: 15
[ ] 2018-10-04 20:05:31: Iteration calculation: 16
[ ] 2018-10-04 20:05:32: Iteration calculation: 17
[ ] 2018-10-04 20:05:34: Iteration calculation: 18
[ ] 2018-10-04 20:05:34: Iteration calcul

[ ] 2018-10-04 20:08:53: Iteration calculation: 160
[ ] 2018-10-04 20:08:53: Iteration calculation: 161
[ ] 2018-10-04 20:08:55: Iteration calculation: 162
[ ] 2018-10-04 20:08:56: Iteration calculation: 163
[ ] 2018-10-04 20:08:56: Iteration calculation: 164
[ ] 2018-10-04 20:08:58: Iteration calculation: 165
[ ] 2018-10-04 20:08:59: Iteration calculation: 166
[ ] 2018-10-04 20:08:59: Iteration calculation: 167
[ ] 2018-10-04 20:09:01: Iteration calculation: 168
[ ] 2018-10-04 20:09:02: Iteration calculation: 169
[ ] 2018-10-04 20:09:03: Iteration calculation: 170
[ ] 2018-10-04 20:09:04: Iteration calculation: 171
[ ] 2018-10-04 20:09:06: Iteration calculation: 172
[ ] 2018-10-04 20:09:07: Iteration calculation: 173
[ ] 2018-10-04 20:09:08: Iteration calculation: 174
[ ] 2018-10-04 20:09:10: Iteration calculation: 175
[ ] 2018-10-04 20:09:11: Iteration calculation: 176
[ ] 2018-10-04 20:09:11: Iteration calculation: 177
[ ] 2018-10-04 20:09:13: Iteration calculation: 178
[ ] 2018-10-

## GWH

In [ ]:
p_gwh = np.array([
    hf.init_guess_by_wolfsberg_helmholtz(mol.get_pyscf_molecule()) for mol in molecules[1]
]).astype("float64")

p_embedded_gwh = np.array(list(map(embedd, zip(p_test, p_gwh))))

In [ ]:
print(make_results_str(measure_all_quantities(
    p_embedded_gwh,
    dataset,
    molecules[1],
    N_ELECTRONS,
    mf_initializer,
    dim,
    is_triu=False,
    is_dataset_triu=True,
    s=S[ind:]
)))

# Test 

In [ ]:
from SCFInitialGuess.utilities.dataset import make_matrix_batch
s_test_batch = make_matrix_batch(S[ind:], dim)

In [ ]:
from SCFInitialGuess.nn.post_processing import multi_mc_wheeny
from SCFInitialGuess.utilities.dataset import make_matrix_batch

#del p_embedded_sad
#del p_sad
#del p_gwh
#del P
#del dataset
#del molecules
#del p_pure
#del s_train
#del p_train

f = lambda x: multi_mc_wheeny(x[0], x[1], 20)

p_embedded_gwh_mcw5 = np.array(list(map(f, zip(p_embedded_gwh, s_test_batch))))

In [ ]:
from SCFInitialGuess.utilities.analysis import mf_initializer

print(make_results_str(measure_all_quantities(
    p_embedded_gwh_mcw5,
    dataset,
    molecules[1],
    N_ELECTRONS,
    mf_initializer,
    dim,
    is_triu=False,
    is_dataset_triu=True,
    s=S[ind:]
)))